In [5]:
from random import randint
import random

# Initial data, this will be converted to binary
courses = 5  # max bin 1001
sections = 2  # max bin 100
teachers = 4  # max bin 11
rooms = 4  # max bin 100  room no 2 and 4 have a capacity of 120 students
classType = 1  # max bin 1 0 for theory, 1 for labtask
maxStudents = 120  # max bin 1111111
timeSlots = 6  # max bin 110
roomSizeMax = 120  # max 120, can be 60
days = 5  # max bin 101


# Initializing list containing sections and their courses
sectionAllocation = {i: [] for i in range(1, sections + 1)}

# Initializing teacherAllocation
teacherAllocation = {i: [] for i in range(1, teachers + 1)}

# Initialize teacherAssignment
teacherAssignment = {i: {j: 0 for j in range(1, courses + 1)} for i in range(1, teachers + 1)}
# Total chromosome population

pop = sections*courses # should be equal section* courses

# initializing chromosome and fitness list
chromosome = []
fitness = {i: 0 for i in range(1, pop + 1)}

def chromosomeInitializer():
    global chromosome
    for section in range(sections):
        # Keeping track of distinct courses each teacher has been assigned
        teacher_course_count = {i: 0 for i in range(1, teachers + 1)}
        for course in range(courses):
            classType = randint(0, 1)
            sectionIndex = section + 1
            students = randint(30, maxStudents)
            teacher = randint(1, teachers)
            day1 = randint(1, days)
            day2 = randint(1, days)
            while day2 == day1 or day2 == day1 + 1:
                day2 = randint(1, days)

            if(classType)==0:
                timeSlot1 = randint(1, 4)
                timeSlot2 = randint(1, 4)
            if(classType)==1:
                timeSlot1 = randint(3, timeSlots)
                timeSlot2 = randint(3, timeSlots)
            room1 = randint(1, rooms)
            roomSize1 = 0
            while roomSize1 < students:
                room1 = randint(1, rooms)
                if room1%2==0:
                    roomSize1 = 120
                else:
                    roomSize1 = 60

            room2=room1
            roomSize2 = 0
            while roomSize2 < students:
                room2 = randint(1, rooms)
                if room2%2==0:
                    roomSize2 = 120
                else:
                    roomSize2 = 60

            if int(classType) == 1:  # lab, so it must have both classes consecutive
                if timeSlot1 == 6:  # last slot
                    timeSlot1 = 5
                    timeSlot2=6
                else:
                    timeSlot2 = timeSlot1 + 1
                room2 = room1
                day2 = day1

            # Check if the teacher can teach more courses and assign the course
            if teacher_course_count[teacher] < 3:
                teacher_course_count[teacher] += 1
                teacherIndex = teacher
            else:
                # Teacher has been assigned max courses.Choose another teacher randomly
                while True:
                    new_teacher = randint(1, teachers)
                    if teacher_course_count[new_teacher] < 3:
                        teacher = new_teacher
                        teacherIndex = teacher
                        teacher_course_count[teacher] += 1
                        break

            # Append course number, section number, and day to teacherAllocation
            teacherAllocation[teacherIndex].append((course + 1, sectionIndex, day1))
            teacherAllocation[teacherIndex].append((course + 1, sectionIndex, day2))

            # Increment teacherAssignment for the course
            teacherAssignment[teacherIndex][course + 1] += 1

            # Applying binary representation with fixed lengths
            course = bin(course + 1)[2:].zfill(len(bin(courses)[2:]))
            classType = bin(classType)[2:].zfill(len(bin(classType)[2:]))
            sectionIndex = bin(sectionIndex)[2:].zfill(len(bin(sections)[2:]))
            students = bin(students)[2:].zfill(len(bin(maxStudents)[2:]))
            teacher = bin(teacher)[2:].zfill(len(bin(teachers)[2:]))
            day1 = bin(day1)[2:].zfill(len(bin(days)[2:]))
            timeSlot1 = bin(timeSlot1)[2:].zfill(len(bin(timeSlots)[2:]))
            room1 = bin(room1)[2:].zfill(len(bin(rooms)[2:]))
            roomSize1 = bin(roomSize1)[2:].zfill(len(bin(roomSizeMax)[2:]))
            day2 = bin(day2)[2:].zfill(len(bin(days)[2:]))
            timeSlot2 = bin(timeSlot2)[2:].zfill(len(bin(timeSlots)[2:]))
            room2 = bin(room2)[2:].zfill(len(bin(rooms)[2:]))
            roomSize2 = bin(roomSize2)[2:].zfill(len(bin(roomSizeMax)[2:]))

            # Concatenate components to form chromosome
            chromosome.append(course + classType + sectionIndex + students + teacher + day1 + timeSlot1 + room1 + roomSize1 + day2 + timeSlot2 + room2 + roomSize2)
            #print(chromosome[-1])

    # Filling sectionAllocation
    for teacher, courses_assigned in teacherAllocation.items():
        for course, section, day in courses_assigned:
            sectionAllocation[section].append((course, teacher, day))
            
            
def calculateFitness(chromosome, population):
    clashes = 0
    gene=chromosome
    for other_chromosome in population:
        
        other_gene = other_chromosome
        # Skip if comparing with itself
        c1 = int(gene[:3], 2)
        c2 = int(other_gene[:3], 2)
        s1 = int(gene[4:6], 2)
        s2 = int(other_gene[4:6], 2)
        if c1 == c2 and s1==s2:
            continue

        # Decode chromosome components
        course = int(gene[:3], 2)
        classType = int(gene[3], 2)
        section = int(gene[4:6], 2)
        teacher = int(gene[13:16], 2)
        strength=int(gene[6:13],2)
        day1 = int(gene[16:19], 2)
        timeSlot1 = int(gene[19:22], 2)
        room1 = int(gene[22:25], 2)
        roomSize1=int(gene[25:32],2)
        # Decode components of other chromosome
        other_section = int(other_gene[4:6], 2)
        other_teacher=int(other_gene[13:16],2)
        other_day1 = int(other_gene[16:19], 2)
        other_timeSlot1 = int(other_gene[19:22], 2)
        other_room1 = int(other_gene[22:25], 2)

        # comparing if teacher schedule has clash
        if(day1==other_day1 and timeSlot1==other_timeSlot1 and teacher==other_teacher):
            clashes += 1
        
        # Compare first time slot of gene with other_chromosome
        if day1 == other_day1 and timeSlot1 == other_timeSlot1 and room1 == other_room1:
            clashes += 1

        # Decode components for the second time slot
        day2 = int(gene[32:35], 2)
        timeSlot2 = int(gene[35:38], 2)
        room2 = int(gene[38:41], 2)

        other_day2 = int(other_gene[32:35], 2)
        other_timeSlot2 = int(other_gene[35:38], 2)
        other_room2 = int(other_gene[38:41], 2)
        roomSize2=int(other_gene[41:48],2)

        if(day2==other_day2 and timeSlot2==other_timeSlot2 and teacher==other_teacher):
            clashes += 1
        # Compare second time slot of gene with other_chromosome
        if day2 == other_day2 and timeSlot2 == other_timeSlot2 and room2 == other_room2:
            clashes += 1
        # classes on adjacent days not allowed
        if(classType != 1 and day1==day2 or day2==day1+1):
            clashes+=1
        if(strength>roomSize1):
            clashes+=1
        if(strength>roomSize2):
            clashes+=1
        if(section==other_section and day1==other_day1 and timeSlot1==other_timeSlot1):
            clashes+=1
        if(section==other_section and day2==other_day2 and timeSlot2==other_timeSlot2):
            clashes+=1
            
        #soft constraints
        if(classType==1 and timeSlot1<3):
            clashes+=1
        if(classType==0 and timeSlot1>4):
            clashes+=1
        if(classType==0 and timeSlot2>4):
            clashes+=1
        if(room1!=room2):
            clashes+=1
        
        #boundary values
        if(timeSlot1>6 or timeSlot1<=0):
            clashes+=1
        if(timeSlot2>6 or timeSlot2<=0):
            clashes+=1
            
        if(room1>4 or room1<=0):
            clashes+=1
        if(room2>4 or room2<=0):
            clashes+=1
            
        if(day1>5 or day1<=0):
            clashes+=1
            
        if(day2>5 or day2<=0):
            clashes+=1
            
    # Fitness is inversely proportional to the number of clashes
    fitness = -clashes
    return fitness

def tournament_selection_worst(fitness, tournament_size):
    selected = []
    while len(selected) < 2:  # Selecting 2 worst chromosomes
        tournament_individuals = random.sample(list(fitness.keys()), tournament_size)
        worst_individuals = sorted(tournament_individuals, key=lambda x: fitness[x])[:2]
        selected.extend(worst_individuals)
    return selected



def crossover(chromosome1,chromosome2):
    point1=randint(16,24) 
    point2 =randint(32,40)
    # Performing crossover
    child1 = chromosome1[:point1] + chromosome2[point1:point2] + chromosome1[point2:]
    child2 = chromosome2[:point1] + chromosome1[point1:point2] + chromosome2[point2:]
    
    
    #validating boundary values
    day=child1[16:19]
    if(int(day,2)<=0 or int(day,2)>5):
        day=bin(1)[2:0].zfill(len(bin(days)[2:]))
        child1=child1[:16]+day+child1[19:]

    timeSlot=child1[19:22]
    if(int(timeSlot,2)<=0 or int(timeSlot,2)>6):
        timeSlot=bin(1)[2:0].zfill(len(bin(timeSlots)[2:]))
        child1=child1[:19]+day+child1[22:]
    
    room1=child1[22:25]
    if(int(room1,2)<=0 or int(room1,2)>4):
        room1=bin(1)[2:0].zfill(len(bin(rooms)[2:]))
        child1=child1[:22]+room1+child1[25:]
    
    roomSize1='0'
    if int(room1) %2==0:
        roomSize1 = bin(120)[2:].zfill(len(bin(roomSizeMax)[2:]))
    else:
        roomSize1 = bin(60)[2:].zfill(len(bin(roomSizeMax)[2:]))
    
    day=child1[32:35]
    if(int(day,2)<=0 or int(day,2)>5):
        day=bin(1)[2:0].zfill(len(bin(days)[2:]))
        child1=child1[:32]+day+child1[35:]

    timeSlot=child1[35:38]
    if(int(timeSlot,2)<=0 or int(timeSlot,2)>6):
        timeSlot=bin(1)[2:0].zfill(len(bin(timeSlots)[2:]))
        child1=child1[:35]+day+child1[38:]
    
    room2=child1[38:41]
    if(int(room2,2)<=0 or int(room2,2)>4):
        room2=bin(1)[2:0].zfill(len(bin(rooms)[2:]))
        child1=child1[:38]+room2+child1[41:]  
    
    roomSize2='0'
    if int(room2) %2==0:
        roomSize2 = bin(120)[2:].zfill(len(bin(roomSizeMax)[2:]))
    else:
        roomSize2 = bin(60)[2:].zfill(len(bin(roomSizeMax)[2:]))
        
    child1=child1[:25]+roomSize1+child1[32:41]+roomSize2 
    
    if child1[3] == '1':
        timeSlot1=child1[19:22]
        if(int(timeSlot1,2)>=6):
            timeSlot1=bin(5)[2:0].zfill(len(bin(timeSlots)[2:]))
            timeSlot2=bin(int(timeSlot1,2)+1)[2:].zfill(len(bin(timeSlots)[2:]))
            child1 = child1[:19]+timeSlot1+child1[22:32] + child1[16:19]+timeSlot2+child1[22:32]
        else:
            timeSlot2=bin(int(timeSlot1,2)+1)[2:].zfill(len(bin(timeSlots)[2:]))
            child1 = child1[:32] + child1[16:19]+timeSlot2+child1[22:32] 
 



    
    day=child2[16:19]
    if(int(day,2)<=0 or int(day,2)>5):
        day=bin(1)[2:0].zfill(len(bin(days)[2:]))
        child2=child2[:16]+day+child2[19:]

    timeSlot=child2[19:22]
    if(int(timeSlot,2)<=0 or int(timeSlot,2)>6):
        timeSlot=bin(1)[2:0].zfill(len(bin(timeSlots)[2:]))
        child2=child2[:19]+day+child2[22:]
    
    room1=child2[22:25]
    if(int(room1,2)<=0 or int(room1,2)>4):
        room1=bin(1)[2:0].zfill(len(bin(rooms)[2:]))
        child2=child2[:22]+room1+child2[25:]
    
    
    roomSize1='0'
    if int(room1) %2==0:
        roomSize1 = bin(120)[2:].zfill(len(bin(roomSizeMax)[2:]))
    else:
        roomSize1 = bin(60)[2:].zfill(len(bin(roomSizeMax)[2:]))
      
    
    day=child2[32:35]
    if(int(day,2)<=0 or int(day,2)>5):
        day=bin(1)[2:0].zfill(len(bin(days)[2:]))
        child2=child2[:32]+day+child2[35:]

    timeSlot=child2[35:38]
    if(int(timeSlot,2)<=0 or int(timeSlot,2)>6):
        timeSlot=bin(1)[2:0].zfill(len(bin(timeSlots)[2:]))
        child2=child2[:35]+day+child2[38:]
    
    room2=child2[38:41]
    if(int(room2,2)<=0 or int(room2,2)>4):
        room2=bin(1)[2:0].zfill(len(bin(rooms)[2:]))
        child2=child2[:38]+room1+child2[41:]
    
    roomSize2='0'
    if int(room2) %2==0:
        roomSize2 = bin(120)[2:].zfill(len(bin(roomSizeMax)[2:]))
    else:
        roomSize2 = bin(60)[2:].zfill(len(bin(roomSizeMax)[2:]))
        
    child2=child2[:25]+roomSize1+child2[32:41]+roomSize2
    
    
    if child2[3] == '1':
        
        timeSlot1=child2[19:22]
        if(int(timeSlot1,2)>=6):
            timeSlot1=bin(5)[2:0].zfill(len(bin(timeSlots)[2:]))
            timeSlot2=bin(int(timeSlot1,2)+1)[2:].zfill(len(bin(timeSlots)[2:]))
            child2 = child2[:19]+timeSlot1+child2[22:32] + child2[16:19]+timeSlot2+child2[22:32] 
        else:
            timeSlot2=bin(int(timeSlot1,2)+1)[2:].zfill(len(bin(timeSlots)[2:]))
            child2 = child2[:32] + child2[16:19]+timeSlot2+child2[22:32]
            
    return child1,child2
   

def mutation(gene):
    # Splitting the gene into components
    course1 = gene[:3]
    classType1 = gene[3]
    sectionIndex1 = gene[4:6]
    students1 = gene[6:13]
    teacher1 = gene[13:16]
    day1 = gene[16:19]
    timeSlot1 = gene[19:22]
    room1 = gene[22:25]
    roomSize1 = gene[25:32]
     
    room1 = bin(randint(1, rooms))[2:].zfill(len(bin(rooms)[2:]))
    day1 = bin(randint(1, days))[2:].zfill(len(bin(days)[2:]))
    if(classType1==0):
        timeSlot1 = bin(randint(1, 4))[2:].zfill(len(bin(timeSlots)[2:]))
    if(classType1==1):
        timeSlot1 = bin(randint(3, timeSlots))[2:].zfill(len(bin(timeSlots)[2:]))
    
    #boundary validations
    if(int(day1,2)<=0 or int(day1,2)>5):
        day1=bin(1)[2:0].zfill(len(bin(days)[2:]))
        
    if(int(timeSlot1,2)<=0 or int(timeSlot1,2)>6):
        timeSlot1=bin(1)[2:0].zfill(len(bin(timeSlots)[2:]))
    
    if(int(room1,2)<=0 or int(room1,2)>4):
        room1=bin(1)[2:0].zfill(len(bin(rooms)[2:]))
    
    if int(room1) %2==0:
        roomSize1 = bin(120)[2:].zfill(len(bin(roomSizeMax)[2:]))
    else:
        roomSize1 = bin(60)[2:].zfill(len(bin(roomSizeMax)[2:]))
    

    day2 = gene[32:35]
    timeSlot2 = gene[35:38]
    room2 = gene[38:41]
    roomSize2 = gene[41:48]
        
    room2 = bin(randint(1, rooms))[2:].zfill(len(bin(rooms)[2:]))
    day2 = bin(randint(1, days))[2:].zfill(len(bin(days)[2:]))
    if(classType1==0):
        timeSlot2 = bin(randint(1, 4))[2:].zfill(len(bin(timeSlots)[2:]))
    if(classType1==1):
        timeSlot2 = bin(randint(3, timeSlots))[2:].zfill(len(bin(timeSlots)[2:]))
    
    if(int(day2,2)<=0 or int(day2,2)>5):
        day2=bin(1)[2:0].zfill(len(bin(days)[2:]))
        
    if(int(timeSlot2,2)<=0 or int(timeSlot2,2)>6):
        timeSlot2=bin(1)[2:0].zfill(len(bin(timeSlots)[2:]))
    
    if(int(room2,2)<=0 or int(room2,2)>4):
        room2=bin(1)[2:0].zfill(len(bin(rooms)[2:]))
    
    if int(room2)%2==0:
        roomSize2 = bin(120)[2:].zfill(len(bin(roomSizeMax)[2:]))
    else:
        roomSize2 = bin(60)[2:].zfill(len(bin(roomSizeMax)[2:]))
    
    mutated_gene=course1+classType1+sectionIndex1+students1+teacher1+day1+timeSlot1+room1+roomSize1+day2+timeSlot2+room2+roomSize2
    if(classType1==1):
        if(int(timeSlot1,2)==6):
            timeSlot1=bin(5)[2:0].zfill(len(bin(timeSlots)[2:]))
            timeSlot2=bin(int(timeSlot1,2)+1)[2:].zfill(len(bin(timeSlots)[2:]))
            mutated_gene = mutated_gene[:19]+timeSlot1+mutated_gene[22:32] + mutated_gene[16:19]+timeSlot2+mutated_gene[22:32]
        else:
            timeSlot2=bin(int(timeSlot1,2)+1)[2:].zfill(len(bin(timeSlots)[2:]))
            mutated_gene = mutated_gene[:32] + mutated_gene[16:19]+timeSlot2+mutated_gene[22:32]
    return mutated_gene

def decodeTimeSlot(timeSlot):
    if timeSlot == '001':
        return '8:30 - 9:50'
    elif timeSlot == '010':
        return '10:00 - 11:20'
    elif timeSlot == '011':
        return '11:30 - 12:50'
    elif timeSlot == '100':
        return '1:00 - 2:20'
    elif timeSlot == '101':
        return '2:30 - 3:50'
    elif timeSlot == '110':
        return '4:00 - 5:20'

def displayTimetable(chromosome, rooms):
    timetable = {day: {timeSlot: {room: [] for room in range(1, rooms + 1)} for timeSlot in range(1, timeSlots + 1)} for day in range(1, days + 1)}

    # Populate timetable with data
    for gene in chromosome:
        # Decode gene components
        course = int(gene[:3], 2)
        classType = int(gene[3], 2)
        sectionIndex = int(gene[4:6], 2)
        students = int(gene[6:13], 2)
        teacher = int(gene[13:16], 2)
        day1 = int(gene[16:19], 2)
        timeSlot1 = int(gene[19:22], 2)
        room1 = int(gene[22:25], 2)
        roomSize1 = int(gene[25:32], 2)
        day2 = int(gene[32:35], 2)
        timeSlot2 = int(gene[35:38], 2)
        room2 = int(gene[38:41], 2)
        roomSize2 = int(gene[41:48], 2)

        # Add course information to the timetable for the first time slot
        timetable[day1][timeSlot1][room1].append((course, sectionIndex, teacher))

        # Add course information to the timetable for the second time slot
        timetable[day2][timeSlot2][room2].append((course, sectionIndex, teacher))

    # Display timetable
    print("Day\tTime Slot\t", end="")
    for timeSlot in range(1, timeSlots + 1):
        print(f"{decodeTimeSlot(format(timeSlot, '03b')):<14}", end="")
    print()

    for day, slots in timetable.items():
        print(f"Day {day}:", end="")
        for room in range(1, rooms + 1):
            print(f"\n\tRoom {room}:", end="")
            for timeSlot in range(1, timeSlots + 1):
                courses = slots.get(timeSlot, {}).get(room, [])
                if courses:
                    course_info = ', '.join([f"C{course},S{section},T{teacher}" for course, section, teacher in courses])
                    print(f"{course_info:<14}", end="")
                else:
                    print("{:<14}".format("-"), end="")
        print("\n")




#initial population
chromosomeInitializer()

all_zero_fitness = False
consecutive_iterations_without_update = 0
previous_fitness_values = None
print("Initial Fitness: ")
for i in range(pop):
    fitness[i + 1] = calculateFitness(chromosome[i], chromosome)
for i, fit in fitness.items():
    print(f"Chromosome {i}: Fitness {fit}")
# Loop until all chromosomes achieve a fitness of 0 or no updates for 15 consecutive iterations
while not all_zero_fitness and consecutive_iterations_without_update < 100:
    # Calculate fitness for each chromosome in the population
    for i in range(pop):
        fitness[i + 1] = calculateFitness(chromosome[i], chromosome)

    # Print fitness for each chromosome
    #for i, fit in fitness.items():
        #print(f"Chromosome {i}: Fitness {fit}")

    all_zero_fitness = all(fit == 0 for fit in fitness.values())

    if all_zero_fitness:
        break

    # Check if fitness values have been updated
    if previous_fitness_values == list(fitness.values()):
        consecutive_iterations_without_update += 1
    else:
        consecutive_iterations_without_update = 0
        previous_fitness_values = list(fitness.values())
        #for i, fit in fitness.items():
            #print(f"Chromosome {i}: Fitness {fit}")

    # Break if no updates for 15 consecutive iterations
    if consecutive_iterations_without_update >= 100:
        break

    tournament_size = 10
    worst_chromosomes = tournament_selection_worst(fitness, tournament_size)
    x, y = worst_chromosomes

    # Perform crossover
    child1, child2 = crossover(chromosome[x - 1], chromosome[y - 1])

    # Calculate fitness for the children
    child1_fitness = calculateFitness(child1, chromosome)
    child2_fitness = calculateFitness(child2, chromosome)

    # Compare children's fitness with parents
    if child1_fitness > fitness[x]:
        chromosome[x - 1] = child1
        fitness[x] = child1_fitness
    else:
        # If child1 is not better, mutate it
        mutated_child1 = mutation(child1)
        mutated_child1_fitness = calculateFitness(mutated_child1, chromosome)
        if mutated_child1_fitness > fitness[x]:
            chromosome[x - 1] = mutated_child1
            fitness[x] = mutated_child1_fitness

    if child2_fitness > fitness[y]:
        chromosome[y - 1] = child2
        fitness[y] = child2_fitness
    else:
        # If child2 is not better, mutate it
        mutated_child2 = mutation(child2)
        mutated_child2_fitness = calculateFitness(mutated_child2, chromosome)
        if mutated_child2_fitness > fitness[y]:
            chromosome[y - 1] = mutated_child2
            fitness[y] = mutated_child2_fitness   

# If the loop ended due to no updates for 15 consecutive iterations, print a message
if consecutive_iterations_without_update >= 15:
    print("No fitness updates for 15 consecutive iterations. Exiting loop.")
else:
    print("All chromosomes have achieved a fitness of 0.")


print("Final Fitness: ")
for i, fit in fitness.items():
    print(f"Chromosome {i}: Fitness {fit}")



# Call the function to display the timetable
displayTimetable(chromosome, rooms)


Initial Fitness: 
Chromosome 1: Fitness -1
Chromosome 2: Fitness -1
Chromosome 3: Fitness -11
Chromosome 4: Fitness -10
Chromosome 5: Fitness -10
Chromosome 6: Fitness -11
Chromosome 7: Fitness -12
Chromosome 8: Fitness -9
Chromosome 9: Fitness -3
Chromosome 10: Fitness -1
All chromosomes have achieved a fitness of 0.
Final Fitness: 
Chromosome 1: Fitness 0
Chromosome 2: Fitness 0
Chromosome 3: Fitness 0
Chromosome 4: Fitness 0
Chromosome 5: Fitness 0
Chromosome 6: Fitness 0
Chromosome 7: Fitness 0
Chromosome 8: Fitness 0
Chromosome 9: Fitness 0
Chromosome 10: Fitness 0
Day	Time Slot	8:30 - 9:50   10:00 - 11:20 11:30 - 12:50 1:00 - 2:20   2:30 - 3:50   4:00 - 5:20   
Day 1:
	Room 1:-             -             -             -             -             -             
	Room 2:-             -             -             C3,S1,T4      -             -             
	Room 3:-             -             -             -             -             -             
	Room 4:C4,S1,T2      -             - 